In [1]:
%cd ~/google_drive/code/projects/sparse/python
from sparse.core.sparse_dataframe import SparseDataFrame
from sparse.utilities.utils import *

%cd ~/google_drive/code/projects/texture_classifier/python
# %cd /home/ubuntu/texture_classifier/python
import multiprocessing
import PIL
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist, squareform
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
import cv2
from pandas.io.pytables import HDFStore
# from plotly import plotly
# from plotly.graph_objs import *

import core.utils
reload(core.utils)
from core.utils import *

import core.image_scanner
reload(core.image_scanner)
from core.image_scanner import ImageScanner

import core.pipeline
reload(core.pipeline)
from core.pipeline import *

/Users/alexbraun/google_drive/code/projects/sparse/python
/Users/alexbraun/google_drive/code/projects/texture_classifier/python


In [2]:
source = '/Users/alexbraun/Documents/data/texture_classifier/data/texture'
spec = [
    'material',
    'image_id',
    'common_name',
    'origin',
    'desc',
    'extension'
]

wood_mask = [
    'moabi',
    'sapele',
#     'olive-ash',
    'european-ash',
    'kingwood',
    'european-lime',
    'african-mahogany',
    'olive'

#     'macassar-ebony',
#     'peruvian-walnut'
#     'bog-oak',
#     'goncalo-alves',
#     'merbau'
]

In [3]:
# random scan
min_res = 10
max_res = 100
params = {
            'scan_method':      'random_scan',
            'min_resolution':   (min_res, min_res),
            'max_resolution':   (max_res, max_res),
            'patches':          100,
            'patch_resolution': (min_res, min_res),
            'normalize':        True,
            'bins':             256
#             'rotation':         'random'
}

# grid scan
# min_res = 400
# max_res = 400
# params = {
#             'scan_method':      'grid_scan',
#             'min_resolution':   (min_res, min_res),
#             'max_resolution':   (max_res, max_res),
#             'resolutions':      1,
#             'spacing':          'even',
# #             'patch_resolution': (min_res, min_res),
#             'normalize':        True,
#             'bins':             256
# }

info = get_info(source, spec)
info = info[info.common_name.apply(lambda x: x in wood_mask)]

# dataframes won't allow direct assignment of dicts
info['params'] = None
info.params = info.params.apply(lambda x: params)
    
# train, test = info_split(info)

test = info[info.desc.apply(lambda x: '_a_' in x)]
train = info[info.origin != 'arroway-textures']

# train = train.head(1)
# test = test.head(1)

In [ ]:
# %%snakeviz

version = '3'.zfill(3)
hdf_path = '/Users/alexbraun/Documents/data/texture_classifier/data/hdf/data'
hdf_path += '.' + version + '.hdf'

%time train_x, valid_x, train_y, valid_y, test_x, test_y = archive_data(train, test, hdf_path)
# archive_data(train, test, hdf_path, cross_val=False)

In [4]:
train = get_data(train, features=['b', 'v'])
train_x, valid_x, train_y, valid_y = train_test_split(train.drop('y', axis=1), train.y, test_size=0.2)

In [8]:
version = '2'.zfill(3)
hdf_path = '/Users/alexbraun/Documents/data/texture_classifier/data/hdf/data'
hdf_path += '.' + version + '.hdf'


hdf = HDFStore(hdf_path)
hdf['train_x'] = DataFrame(train_x)
hdf['valid_x'] = DataFrame(valid_x)
hdf['train_y'] = Series(train_y)
hdf['valid_y'] = Series(valid_y)
hdf.close()

In [ ]:
clf = SVC()
params = {
    'C':            [5], #np.arange(0.1, 1, 0.1),
    'kernel':       ['linear'],#, 'rbf'],
#     'degree':       [3],
#     'gamma':        [0.0],
#     'coef0':        [0.0],
#     'shrinking':    [True],
#     'probability':  [False],
#     'tol':          [0.001],
#     'cache_size':   [200],
#     'class_weight': [None],
#     'verbose':      [False],
#     'max_iter':     [1],
#     'random_state': [None]
}

grid = GridSearchCV(clf, params, cv=5)
%time grid.fit(train_x, train_y)
pred = grid.best_estimator_.predict(valid_x)
valid_report = get_report(valid_y, pred)
# pred = grid.best_estimator_.predict(test_x)
# print(grid.best_params_)
# print(grid.best_estimator_.score(test_x, test_y))
# report = get_report(test_y, pred)
# report
valid_report

In [ ]:
d = DataFrame([valid_y, pred]).T
d.columns = ['ytrue', 'yhat']
lut = {k:i for i, k in enumerate(wood_mask)}
ilut = {v:k for k, v in lut.iteritems()}
d.yhat = d.yhat.apply(lambda x: lut[x])
d = d.groupby('ytrue').agg(lambda x: x.mode()).yhat.apply(lambda x: ilut[x])
d = DataFrame(d)
print(d[d.index == d.yhat].shape[0] / float(d.shape[0]))
d